In [1]:
import time
import numpy as np
import datajoint as dj
import networkx as nx

In [2]:
#configure datajoint access to skeletons
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")

Connecting celiib@10.28.0.34:3306


In [3]:
#just gets the ids and the edges
neurite_skeletons = (dj.U("segment_id","edges") 
                     & ta3p100.NeuriteRawSkeleton).fetch(
                        as_dict=True)



In [5]:
#process Example of the filtered skeletons
example_skeleton = neurite_skeletons[2]

#unpacks so just list of vertices
vertices_unpacked  = example_skeleton["edges"].reshape(-1,3)
#reduce the number of repeat vertices and convert to list
unique_rows = np.unique(vertices_unpacked, axis=0)
unique_rows_list = unique_rows.tolist()

#assigns the number to the vertex (in the original vertex list) that corresponds to the index in the unique list
vertices_unpacked_coefficients = [unique_rows_list.index(a) for a in vertices_unpacked.tolist()]

#reshapes the vertex list to become an edge list
edges_with_coefficients =  np.array(vertices_unpacked_coefficients).reshape(-1,2)
len(edges_with_coefficients),len(example_skeleton["edges"])

#create the graph from the edges
B = nx.Graph()
B.add_nodes_from([(x,{"coordinates":y}) for x,y in enumerate(unique_rows_list)])
B.add_edges_from(edges_with_coefficients)
#possible_segmentations = [n for n,x in B.degree() if x > 3]
#possible_segmentations

In [18]:
help(np.linalg.norm)

Help on function norm in module numpy.linalg:

norm(x, ord=None, axis=None, keepdims=False)
    Matrix or vector norm.
    
    This function is able to return one of eight different matrix norms,
    or one of an infinite number of vector norms (described below), depending
    on the value of the ``ord`` parameter.
    
    Parameters
    ----------
    x : array_like
        Input array.  If `axis` is None, `x` must be 1-D or 2-D.
    ord : {non-zero int, inf, -inf, 'fro', 'nuc'}, optional
        Order of the norm (see table under ``Notes``). inf means numpy's
        `inf` object.
    axis : {int, 2-tuple of ints, None}, optional
        If `axis` is an integer, it specifies the axis of `x` along which to
        compute the vector norms.  If `axis` is a 2-tuple, it specifies the
        axes that hold 2-D matrices, and the matrix norms of these matrices
        are computed.  If `axis` is None then either a vector norm (when `x`
        is 1-D) or a matrix norm (when `x` is 2-D) i

In [19]:
node_1_xyz = np.array(B.nodes[0]["coordinates"])
node_2_xyz = np.array(B.nodes[1]["coordinates"])

dist = np.linalg.norm(node_1_xyz-node_2_xyz)
print(dist)
threshold = 1500
if dist < threshold:
    print("too close")

710.3872183534836
too close


In [6]:
from tqdm import tqdm
possible_error_neurites = []

for skeleton in tqdm(neurite_skeletons):
    #print(str(skeleton['segment_id']) + ":")
    
    vertices_unpacked  = skeleton["edges"].reshape(-1,3)
    if not vertices_unpacked.any():
        continue
    #reduce the 
    unique_rows = np.unique(vertices_unpacked, axis=0)
    unique_rows_list = unique_rows.tolist()
    #assigns the number to the vertex that corresponds to the index in the unique list
    vertices_unpacked_coefficients = [unique_rows_list.index(a) for a in vertices_unpacked.tolist()]
    edges_with_coefficients =  np.array(vertices_unpacked_coefficients).reshape(-1,2)
    #len(edges_with_coefficients),len(example_skeleton["edges"])
    
    B = nx.Graph()
    B.add_nodes_from([(x,{"coordinates":y}) for x,y in enumerate(unique_rows_list)])
    B.add_edges_from(edges_with_coefficients)
    possible_segmentations = [n for n,x in B.degree() if x > 3]
    if len(possible_segmentations) > 0:
        possible_error_neurites.append({skeleton['segment_id']:len(possible_segmentations)})
          
          
    

100%|██████████| 32234/32234 [04:12<00:00, 127.73it/s]


In [7]:
print(len(neurite_skeletons),len(possible_error_neurites))

32234 4590


In [8]:
4590/32234

0.142396227585779

In [29]:
from tqdm import tqdm
possible_error_neurites_2 = []

tree_threshold = 50

for skeleton in tqdm(neurite_skeletons):
    #print(str(skeleton['segment_id']) + ":")
    
    vertices_unpacked  = skeleton["edges"].reshape(-1,3)
    if not vertices_unpacked.any():
        continue
    #reduce the 
    unique_rows = np.unique(vertices_unpacked, axis=0)
    unique_rows_list = unique_rows.tolist()
    #assigns the number to the vertex that corresponds to the index in the unique list
    vertices_unpacked_coefficients = [unique_rows_list.index(a) for a in vertices_unpacked.tolist()]
    edges_with_coefficients =  np.array(vertices_unpacked_coefficients).reshape(-1,2)
    #len(edges_with_coefficients),len(example_skeleton["edges"])
    
    B = nx.Graph()
    B.add_nodes_from([(x,{"coordinates":y}) for x,y in enumerate(unique_rows_list)])
    B.add_edges_from(edges_with_coefficients)
    possible_segmentations_4 = [n for n,x in B.degree() if x > 3]
    if len(possible_segmentations_4) > 0:
        possible_error_neurites_2.append({skeleton['segment_id']:"4 split"})
        
    else:
        possible_segmentations_3 = [n for n,x in B.degree() if x > 2]
        
        
        if len(possible_segmentations_3) > 1:
            break_Flag = False
            for ps in possible_segmentations_3:
                if break_Flag == True:
                    break
                for ps_2 in possible_segmentations_3:
                    if break_Flag == True:
                        break
                    if ps != ps_2:
                        #check the distance of the points
                        node_1_xyz = np.array(B.nodes[ps]["coordinates"])
                        node_2_xyz = np.array(B.nodes[ps_2]["coordinates"])
                        
                        dist = np.linalg.norm(node_1_xyz-node_2_xyz)
                        #print(dist)
                        
                        if dist < tree_threshold:
                            possible_error_neurites_2.append({skeleton['segment_id']:"3 split"})
                            break_Flag = True
                            break
                        
                        
                        
          


100%|██████████| 32234/32234 [04:31<00:00, 118.78it/s]

In [25]:

#WITH THRESHOLD = 500
print(len(neurite_skeletons),len(possible_error_neurites),len(possible_error_neurites_2))

32234 4590 6305


In [26]:
6305/32234

0.19560091828504064

In [28]:
#examples to plot meshes
possible_error_neurites_2[1]
possible_error_neurites[1]

{648518346341351718: 2}

In [30]:

#WITH THRESHOLD = 50
print(len(neurite_skeletons),len(possible_error_neurites),len(possible_error_neurites_2))

32234 4590 4590
